In [31]:
import pywt
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.graph_objects as go
import gc
import warnings

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

warnings.filterwarnings("ignore")
%matplotlib inline
import plotly.express as px
#import pandas_profiling as pp

In [3]:
hw1_cost = 450

In [4]:
tesla_data = pd.read_csv('tesla_vehicles.csv')
tesla_data

,Year,Quarter,Months,Total Vehicles,New Vehicles,Roadster,Model S,Model X,Model 3,Autopilot Hardware,autopiolot-stand,autopiolot-enhance,model3_option_autopilot,Automotive revenues,Automotive gross profit,cost,headcount,average cost,develop cost,ic cost
0,2008,2,2008-Q2,3,3,3.0,NaN,NaN,NaN,0,NaN,NaN,0.0,0,0,0,NaN,NaN,NaN,NaN
1,2008,3,2008-Q3,30,27,27.0,NaN,NaN,NaN,0,NaN,NaN,0.0,0,0,0,NaN,NaN,NaN,NaN
2,2008,4,2008-Q4,100,70,70.0,NaN,NaN,NaN,0,NaN,NaN,0.0,0,0,0,NaN,NaN,NaN,NaN
3,2009,1,2009-Q1,320,220,220.0,NaN,NaN,NaN,0,NaN,NaN,0.0,0,0,0,NaN,NaN,NaN,NaN
4,2009,2,2009-Q2,500,180,180.0,NaN,NaN,NaN,0,NaN,NaN,0.0,0,0,0,NaN,NaN,NaN,NaN
5,2009,3,2009-Q3,700,200,200.0,NaN,NaN,NaN,0,NaN,NaN,0.0,0,0,0,NaN,NaN,NaN,NaN
6,2009,4,2009-Q4,1000,300,300.0,NaN,NaN,NaN,0,NaN,NaN,0.0,0,0,0,NaN,NaN,NaN,NaN
7,2010,1,2010-Q1,1100,100,100.0,NaN,NaN,NaN,0,NaN,NaN,0.0,0,0,0,NaN,NaN,NaN,NaN
8,2010,2,2010-Q2,1200,100,100.0,NaN,NaN,NaN,0,NaN,NaN,0.0,0,0,0,NaN,NaN,NaN,NaN
9,2010,3,2010-Q3,1300,100,100.0,NaN,NaN,NaN,0,NaN,NaN,0.0,0,0,0,NaN,NaN,NaN,NaN


In [5]:
tesla_data['Automotive revenues'] = tesla_data['Automotive revenues'].apply(lambda x: int(x.replace(',','')))
tesla_data['Automotive gross profit'] = tesla_data['Automotive gross profit'].apply(lambda x: int(x.replace(',','')))
tesla_data['Total Vehicles'] = tesla_data['Total Vehicles'].apply(int)

In [6]:
tesla_data_hw0 = tesla_data.copy()
tesla_data_hw1 = tesla_data.copy()
tesla_data_hw2 = tesla_data.copy()
tesla_data_hw3 = tesla_data.copy()

In [7]:
tesla_data_hw0.loc[tesla_data['Autopilot Hardware'] > 0, 'Total Vehicles'] = tesla_data[tesla_data['Autopilot Hardware'] == 0]['Total Vehicles'].max()
tesla_data_hw1.loc[tesla_data['Autopilot Hardware'] > 1, 'Total Vehicles'] = tesla_data[tesla_data['Autopilot Hardware'] == 1]['Total Vehicles'].max()
tesla_data_hw1.loc[tesla_data['Autopilot Hardware'] < 1, 'Total Vehicles'] = 0
tesla_data_hw2.loc[tesla_data['Autopilot Hardware'] > 2, 'Total Vehicles'] = tesla_data[tesla_data['Autopilot Hardware'] == 2]['Total Vehicles'].max()
tesla_data_hw2.loc[tesla_data['Autopilot Hardware'] < 2, 'Total Vehicles'] = 0
tesla_data_hw3.loc[tesla_data['Autopilot Hardware'] < 3, 'Total Vehicles'] = 0

In [9]:
a0 = go.Bar(x=tesla_data_hw0['Months'].values, y=tesla_data_hw0['Total Vehicles'].values,name = 'No Autopilot Hardware')
a = go.Bar(x=tesla_data_hw1['Months'].values, y=tesla_data_hw1['Total Vehicles'].values,name = 'Autopilot Hardware 1')
b = go.Bar(x=tesla_data_hw2['Months'].values, y=tesla_data_hw2['Total Vehicles'].values,name = 'Autopilot Hardware 2+')
c = go.Bar(x=tesla_data_hw3['Months'].values, y=tesla_data_hw3['Total Vehicles'].values,name = 'Autopilot Hardware 3')
fig = go.Figure(data=[c,b,a,a0])
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
#fig.update_traces(opacity=0.75)

fig.show()

In [10]:
revenues = go.Bar(x=tesla_data['Months'].values, y=tesla_data['Automotive revenues'].values,name = 'Automotive revenues')
gross = go.Bar(x=tesla_data['Months'].values, y=tesla_data['Automotive gross profit'].values,name = 'Automotive gross profit')
new_car = go.Bar(x=tesla_data['Months'].values, y=tesla_data['New Vehicles'].values/10,name = 'New Vehicles')
fig = go.Figure(data=[revenues, gross, new_car])
fig.show()

In [12]:
tesla_data.loc[tesla_data['Autopilot Hardware'] <= 2, 'hardware_income'] = tesla_data['New Vehicles'] *tesla_data['autopiolot-stand'] *5000 \
                                        + tesla_data['New Vehicles'] *tesla_data['autopiolot-enhance'] *3000
tesla_data.loc[tesla_data['Autopilot Hardware'] == 3, 'hardware_income'] = (tesla_data['New Vehicles']- tesla_data['Model 3'])*tesla_data['autopiolot-stand'] *7000 \
                                        + tesla_data['model3_option_autopilot'] *tesla_data['Model 3']*7000

In [14]:
fig = px.line(x=tesla_data.Months.values, y=tesla_data.hardware_income.values, labels={'x':'时间','y':'收入'})

fig.show()

In [15]:
income = go.Scatter(x=tesla_data.Months.values, y=tesla_data.hardware_income.values/1000000, name='ap income')
revenues = go.Bar(x=tesla_data['Months'].values, y=tesla_data['Automotive revenues'].values,name = 'Automotive revenues')
gross = go.Bar(x=tesla_data['Months'].values, y=tesla_data['Automotive gross profit'].values,name = 'Automotive gross profit')
new_car = go.Bar(x=tesla_data['Months'].values, y=tesla_data['New Vehicles'].values/10,name = 'New Vehicles')
fig = go.Figure(data=[revenues, gross, new_car,income])
fig.show()

In [21]:
# tesla_data['vehicles_cost_quarter'] = tesla_data['New Vehicles']*tesla_data['cost']*tesla_data['autopiolot-stand'] + tesla_data['New Vehicles']*hw1_cost*(1-tesla_data['autopiolot-stand'])
# tesla_data['develop_cost_quarter'] = (tesla_data['headcount']*tesla_data['average cost'] + tesla_data['develop cost'] + tesla_data['ic cost'])*10000

tesla_data.loc[tesla_data['Model 3'].isna() == True, 'vehicles_cost_quarter'] = tesla_data['New Vehicles']*tesla_data['cost']*tesla_data['autopiolot-stand'] + tesla_data['New Vehicles']*hw1_cost*(1-tesla_data['autopiolot-stand'])


tesla_data.loc[tesla_data['Model 3'].isna() == False, 'vehicles_cost_quarter'] = \
            (tesla_data['New Vehicles'] - tesla_data['Model 3'])*tesla_data['cost']*tesla_data['autopiolot-stand'] \
            + tesla_data['Model 3']*tesla_data['cost']*tesla_data['model3_option_autopilot'] \
             + tesla_data['Model 3']*hw1_cost*(1-tesla_data['model3_option_autopilot']) \
             + (tesla_data['New Vehicles'] - tesla_data['Model 3'])*hw1_cost*(1-tesla_data['autopiolot-stand'])
tesla_data['develop_cost_quarter'] = (tesla_data['headcount']*tesla_data['average cost'] + tesla_data['develop cost'] + tesla_data['ic cost'])*10000


In [22]:
tesla_data['autopilot_gross'] = tesla_data['hardware_income'] - tesla_data['vehicles_cost_quarter']

In [23]:
tesla_data['autopilot_net'] = tesla_data['autopilot_gross'] - tesla_data['develop_cost_quarter']

In [24]:
sum_autopilot_net = 0
for i in range(len(tesla_data)):
    line = tesla_data.iloc[i]
    if np.isnan(line['autopilot_net']) == False:
        sum_autopilot_net += line.autopilot_net
        tesla_data.loc[i, 'total_autopilot_net'] = sum_autopilot_net

In [25]:
tesla_data

,Year,Quarter,Months,Total Vehicles,New Vehicles,Roadster,Model S,Model X,Model 3,Autopilot Hardware,...,headcount,average cost,develop cost,ic cost,hardware_income,vehicles_cost_quarter,autopilot_gross,develop_cost_quarter,autopilot_net,total_autopilot_net
0,2008,2,2008-Q2,3,3,3.0,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008,3,2008-Q3,30,27,27.0,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2008,4,2008-Q4,100,70,70.0,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009,1,2009-Q1,320,220,220.0,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009,2,2009-Q2,500,180,180.0,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2009,3,2009-Q3,700,200,200.0,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2009,4,2009-Q4,1000,300,300.0,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2010,1,2010-Q1,1100,100,100.0,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2010,2,2010-Q2,1200,100,100.0,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2010,3,2010-Q3,1300,100,100.0,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
net = go.Bar(x=tesla_data.Months.values, y=tesla_data.autopilot_net.values,name = 'autopilot_net')
autopilot_gross = go.Bar(x=tesla_data['Months'].values, y=tesla_data['autopilot_gross'].values,name = 'autopilot_gross')
abs_net = go.Scatter(x=tesla_data.Months.values, y=tesla_data.total_autopilot_net.values,name = 'autopilot_abs_net')
# gross = go.Bar(x=tesla_data['Months'].values, y=tesla_data['Automotive gross profit'].values,name = 'Automotive gross profit')
# new_car = go.Bar(x=tesla_data['Months'].values, y=tesla_data['New Vehicles'].values/10,name = 'New Vehicles')
fig = go.Figure(data=[net, autopilot_gross,abs_net])
fig.show()